In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/uwie'

role = sagemaker.get_execution_role()

In [2]:
inputs = sagemaker_session.upload_data(path='Dataset', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-641697043155/sagemaker/uwie


In [2]:
!pygmentize train.py

import time
from data import create_dataset
from model import create_model
from options.TrainOptions import TrainOptions
from utils.TrainStats import TrainStats
from utils.setup_cloud import setup_cloud

if __name__ == '__main__':
    opt = TrainOptions().parse()
    opt = setup_cloud(opt)

    dataset = create_dataset(dataroot=opt.training_data_dir, subdir=opt.subdir, phase=opt.phase,
                             serial_batches=opt.serial_batches, preprocess=opt.preprocess, no_flip=opt.no_flip,
                             load_size=opt.load_size, crop_size=opt.crop_size, batch_size=opt.batch_size,
                             is_distributed=opt.is_distributed, use_cuda=opt.use_cuda)

    dataset_size = len(dataset)
    print('The number of training images = %d' % dataset_size)

    model = create_model(opt)
    stats = TrainStats(opt)
    

    # Training
    for epoch in range(opt.epoch_count, opt.n_epochs + opt.n_epochs_decay + 1):
        print(f"Training {epoch}/{opt.n_epochs + o

In [3]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    role=role,
                    source_dir='.',
                    py_version='py3',
                    framework_version='1.8.1',
                    instance_count=2,
                    instance_type='ml.g4dn.xlarge',
                    hyperparameters={
                        'model':'cyclegan2',
                        'preprocess': 'RRC',
                        'n_epochs': 20,
                        'n_epochs_decay': 0,
                        'backend': 'nccl',
                        'batch-size': 8
                    },metric_definitions=[
                        {'Name': 'loss:D_A', 'Regex': 'loss_D_A=(.*?);'},
                        {'Name': 'loss:D_B', 'Regex': 'loss_D_B=(.*?);'},
                        {'Name': 'loss:G_AtoB', 'Regex': 'loss_G_AtoB=(.*?);'},
                        {'Name': 'loss:G_BtoA', 'Regex': 'loss_G_BtoA=(.*?);'},
                        {'Name': 'cycle_loss:A', 'Regex': 'cycle_loss_A=(.*?);'},
                        {'Name': 'cycle_loss:B', 'Regex': 'cycle_loss_B=(.*?);'},
                    ])

In [4]:
estimator.fit({'training': 's3://sagemaker-us-east-1-641697043155/sagemaker/uwie'},wait=False)